<a href="https://colab.research.google.com/github/martintmv-git/RB-IBDM/blob/main/GSL/gsl_script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GSL
#### Image processing script

<img src="https://i.imgur.com/WY36h4Q.png"
alt="hfspace-insectsam" width="800"/>


## You will need:
- Hugging Face 🤗 Access Token https://huggingface.co/settings/tokens

- Name = HF_TOKEN
- Value = hf_xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx

## Set-up environment

In [1]:
!nvidia-smi # feel free to delete this, it's there to ensure we are using CUDA

Sun Jun 23 16:04:57 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
from google.colab import drive # only if Colab is used
drive.mount('/content/drive')

Mounted at /content/drive


# Install environment requirements

In [3]:
#GroundedSAM
%cd /content
!git clone https://github.com/IDEA-Research/Grounded-Segment-Anything
%cd /content/Grounded-Segment-Anything
!pip install -q -r requirements.txt
%cd /content/Grounded-Segment-Anything/GroundingDINO
!pip install -q .
%cd /content/Grounded-Segment-Anything/segment_anything
!pip install -q .
%cd /content/Grounded-Segment-Anything
#LaVa
!pip install -q simple-lama-inpainting

/content
Cloning into 'Grounded-Segment-Anything'...
remote: Enumerating objects: 1766, done.
remote: Total 1766 (delta 0), reused 0 (delta 0), pack-reused 1766
Receiving objects: 100% (1766/1766), 124.79 MiB | 30.83 MiB/s, done.
Resolving deltas: 100% (821/821), done.
/content/Grounded-Segment-Anything
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 86.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.0/124.0 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 87.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.7/254.7 kB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.3/266.3 kB 25.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... do

# Add imports

In [4]:
import os, sys

sys.path.append(os.path.join(os.getcwd(), "GroundingDINO"))

import argparse
import copy
import time

from IPython.display import display
import PIL
from PIL import Image, ImageDraw, ImageFont, ImageChops, ImageEnhance
from torchvision.ops import box_convert

# GroundingDINO
import GroundingDINO.groundingdino.datasets.transforms as T
from GroundingDINO.groundingdino.models import build_model
from GroundingDINO.groundingdino.util import box_ops
from GroundingDINO.groundingdino.util.slconfig import SLConfig
from GroundingDINO.groundingdino.util.utils import clean_state_dict, get_phrases_from_posmap
from GroundingDINO.groundingdino.util.inference import annotate, load_image, predict

from huggingface_hub import hf_hub_download

import supervision as sv
from supervision import BoundingBoxAnnotator, LabelAnnotator

# SAM
from segment_anything import build_sam, SamPredictor
import cv2
import numpy as np
import matplotlib.pyplot as plt

# LaMa
import requests
import torch
from io import BytesIO
from simple_lama_inpainting import SimpleLama

# Load the models

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## GroundingDINO

In [6]:
def load_model_hf(repo_id, filename, ckpt_config_filename, device='cpu'):
    cache_config_file = hf_hub_download(repo_id=repo_id, filename=ckpt_config_filename)

    args = SLConfig.fromfile(cache_config_file)
    args.device = device
    model = build_model(args)

    cache_file = hf_hub_download(repo_id=repo_id, filename=filename)
    checkpoint = torch.load(cache_file, map_location=device)
    log = model.load_state_dict(clean_state_dict(checkpoint['model']), strict=False)
    print("Model loaded from {} \n => {}".format(cache_file, log))
    _ = model.eval()
    return model

In [7]:
ckpt_repo_id = "ShilongLiu/GroundingDINO"
ckpt_filenmae = "groundingdino_swinb_cogcoor.pth"
ckpt_config_filename = "GroundingDINO_SwinB.cfg.py"


groundingdino_model = load_model_hf(ckpt_repo_id, ckpt_filenmae, ckpt_config_filename, device)

GroundingDINO_SwinB.cfg.py:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

final text_encoder_type: bert-base-uncased


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

groundingdino_swinb_cogcoor.pth:   0%|          | 0.00/938M [00:00<?, ?B/s]

Model loaded from /root/.cache/huggingface/hub/models--ShilongLiu--GroundingDINO/snapshots/a94c9b567a2a374598f05c584e96798a170c56fb/groundingdino_swinb_cogcoor.pth 
 => _IncompatibleKeys(missing_keys=[], unexpected_keys=['label_enc.weight', 'bert.embeddings.position_ids'])


## SAM

In [8]:
! wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth

sam_checkpoint = 'sam_vit_h_4b8939.pth'

sam_predictor = SamPredictor(build_sam(checkpoint=sam_checkpoint).to(device))

--2024-06-23 16:13:04--  https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 18.155.173.40, 18.155.173.80, 18.155.173.79, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|18.155.173.40|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2564550879 (2.4G) [binary/octet-stream]
Saving to: ‘sam_vit_h_4b8939.pth’

sam_vit_h_4b8939.pt 100%[===================>]   2.39G  78.7MB/s    in 29s     

2024-06-23 16:13:33 (83.1 MB/s) - ‘sam_vit_h_4b8939.pth’ saved [2564550879/2564550879]



## LaMa

In [9]:
simple_lama = SimpleLama()

Downloading: "https://github.com/enesmsahin/simple-lama-inpainting/releases/download/v0.1.0/big-lama.pt" to /root/.cache/torch/hub/checkpoints/big-lama.pt
100%|██████████| 196M/196M [00:00<00:00, 338MB/s]


# Inference

In [10]:
def detect(image, image_source, model, text_prompt='insect . flower . cloud', box_threshold=0.15, text_threshold=0.15):
    boxes, logits, phrases = predict(
        image=image,
        model=model,
        caption=text_prompt,
        box_threshold=box_threshold,
        text_threshold=box_threshold
    )
    annotated_frame = annotate(image_source=image_source, boxes=boxes, logits=logits, phrases=phrases)

    # Ensure annotated_frame is a numpy array of uint8
    if not isinstance(annotated_frame, np.ndarray):
        if isinstance(annotated_frame, Image.Image):
            annotated_frame = np.array(annotated_frame)
            print("Converted PIL Image to numpy array")
        else:
            annotated_frame = np.array(annotated_frame, dtype=np.uint8)
            print("Converted annotated_frame to numpy array with uint8 dtype")

    if annotated_frame.dtype != np.uint8:
        annotated_frame = annotated_frame.astype(np.uint8)
        print("Converted annotated_frame to uint8 dtype")

    annotated_frame = annotated_frame[...,::-1] # BGR to RGB
    return annotated_frame, boxes, phrases

def segment(image, sam_model, boxes):
    sam_model.set_image(image)
    H, W, _ = image.shape
    boxes_xyxy = box_ops.box_cxcywh_to_xyxy(boxes) * torch.Tensor([W, H, W, H])
    transformed_boxes = sam_model.transform.apply_boxes_torch(boxes_xyxy.to(device), image.shape[:2])
    masks, _, _ = sam_model.predict_torch(
        point_coords=None,
        point_labels=None,
        boxes=transformed_boxes,
        multimask_output=True,
    )
    return masks.cpu()

def draw_mask(mask, image, random_color=True):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.8])], axis=0)
    else:
        color = np.array([30/255, 144/255, 255/255, 0.6])
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    annotated_frame_pil = Image.fromarray(image).convert("RGBA")
    mask_image_pil = Image.fromarray((mask_image.cpu().numpy() * 255).astype(np.uint8)).convert("RGBA")
    return np.array(Image.alpha_composite(annotated_frame_pil, mask_image_pil))

def dilate_mask(mask, dilate_factor=15):
    mask = mask.astype(np.uint8)
    mask = cv2.dilate(mask, np.ones((dilate_factor, dilate_factor), np.uint8), iterations=1)
    return mask

def process_image(image_path, output_path):
    start_time = time.time()
    try:
        print(f"Processing image: {image_path}")

        # Load image
        image_source, image = load_image(image_path)

        # GroundingDINO
        annotated_frame, detected_boxes, phrases = detect(image, image_source, model=groundingdino_model)

        # SAM
        indices = [i for i, s in enumerate(phrases) if 'insect' in s]
        segmented_frame_masks = segment(image_source, sam_predictor, boxes=detected_boxes[indices])

        # Combine all masks into one for easy visualization
        final_mask = None
        for i in range(len(segmented_frame_masks) - 1):
            if final_mask is None:
                final_mask = np.bitwise_or(segmented_frame_masks[i][0].cpu(), segmented_frame_masks[i + 1][0].cpu())
            else:
                final_mask = np.bitwise_or(final_mask, segmented_frame_masks[i + 1][0].cpu())

        annotated_frame_with_mask = draw_mask(final_mask, image_source)

        # LaMa
        mask = final_mask.numpy().astype(np.uint8) * 255
        mask = dilate_mask(mask)
        dilated_image_mask_pil = Image.fromarray(mask)

        result = simple_lama(image_source, dilated_image_mask_pil)

        # Difference calculation and saving result
        img1 = Image.fromarray(image_source)
        img2 = result

        diff = ImageChops.difference(img2, img1)
        threshold = 7
        diff2 = diff.convert('L').point(lambda p: 255 if p > threshold else 0).convert('1')
        img3 = Image.new('RGB', img1.size, (255, 236, 10))
        diff3 = Image.composite(img1, img3, diff2)

        diff3.save(output_path)
        print(f"Successfully processed and saved: {output_path}")
    except Exception as e:
        print(f"Failed to process image {image_path}: {e}")
    end_time = time.time()
    print(f"Processing time for {image_path}: {end_time - start_time:.2f} seconds")

# Batch processing
input_folder = '/content/drive/MyDrive/diopsis-test-images'
output_folder = '/content/drive/MyDrive/diopsis-test-images-output'

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

for filename in os.listdir(input_folder):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        input_path = os.path.join(input_folder, filename)
        output_path = os.path.join(output_folder, f"{os.path.splitext(filename)[0]}_processed.jpg")
        process_image(input_path, output_path)

Processing image: /content/drive/MyDrive/diopsis-test-images/demo.jpg


SupervisionWarnings: annotate is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead


Successfully processed and saved: /content/drive/MyDrive/diopsis-test-images-output/demo_processed.jpg
Processing time for /content/drive/MyDrive/diopsis-test-images/demo.jpg: 19.25 seconds
Processing image: /content/drive/MyDrive/diopsis-test-images/demo1.jpg


SupervisionWarnings: annotate is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead


Successfully processed and saved: /content/drive/MyDrive/diopsis-test-images-output/demo1_processed.jpg
Processing time for /content/drive/MyDrive/diopsis-test-images/demo1.jpg: 12.67 seconds
Processing image: /content/drive/MyDrive/diopsis-test-images/demo2.jpg


...

Successfully processed and saved: /content/drive/MyDrive/diopsis-test-images-output/demo3_processed.jpg
Processing time for /content/drive/MyDrive/diopsis-test-images/demo3.jpg: 4.53 seconds
Processing image: /content/drive/MyDrive/diopsis-test-images/demo4.jpg


SupervisionWarnings: annotate is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead


Successfully processed and saved: /content/drive/MyDrive/diopsis-test-images-output/demo4_processed.jpg
Processing time for /content/drive/MyDrive/diopsis-test-images/demo4.jpg: 4.67 seconds
Processing image: /content/drive/MyDrive/diopsis-test-images/demo5.jpg


SupervisionWarnings: annotate is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead


Successfully processed and saved: /content/drive/MyDrive/diopsis-test-images-output/demo5_processed.jpg
Processing time for /content/drive/MyDrive/diopsis-test-images/demo5.jpg: 4.91 seconds
